In [1]:
import numpy as np

from reader import *
from encoder import *

In [2]:
bunny_model = Reader.read_from_file('assets/bunny.obj')

In [3]:
def print_statistics(compressed_model, name):
    print(f"{name} statistics:")
    print(f"- Bytes per triangle: {compressed_model.bits_per_triangle / 8:.2f}")
    print(f"- Bytes per vertex: {compressed_model.bits_per_vertex / 8:.2f}")

In [4]:
baseline_encoder = BaselineEncoder()
compressed_model = baseline_encoder.encode(bunny_model)

print_statistics(compressed_model, "BaselineEncoder")

BaselineEncoder statistics:
- Bytes per triangle: 18.05
- Bytes per vertex: 35.82


In [5]:
encoder_no_packing = SimpleQuantizator(pack_strip=Packing.NONE, verbose=True)
compressed_model = encoder_no_packing.encode(bunny_model)

print_statistics(compressed_model, "SimpleQuantizator(no packing)")

SimpleQuantizator verbose statistics:
- Header (in bytes): 32
- Triangles strip (in bytes): 46344
- Quantized vertices (in bytes): 11265
SimpleQuantizator(no packing) statistics:
- Bytes per triangle: 11.60
- Bytes per vertex: 23.03


In [6]:
encoder_fixed_packing = SimpleQuantizator(pack_strip=Packing.FIXED, verbose=True)
compressed_model = encoder_fixed_packing.encode(bunny_model)

print_statistics(compressed_model, "SimpleQuantizator(fixed packing)")

SimpleQuantizator verbose statistics:
- Header (in bytes): 32
- Triangles strip (in bytes): 17379
- Quantized vertices (in bytes): 11265
SimpleQuantizator(fixed packing) statistics:
- Bytes per triangle: 5.77
- Bytes per vertex: 11.46


In [7]:
encoder_binary_range_partitioning_packing = SimpleQuantizator(pack_strip=Packing.BINARY_RANGE_PARTITIONING, verbose=True)
compressed_model = encoder_binary_range_partitioning_packing.encode(bunny_model)

print_statistics(compressed_model, "SimpleQuantizator(binary range partitioning packing)")

SimpleQuantizator verbose statistics:
- Header (in bytes): 32
- Triangles strip (in bytes): 16958
- Quantized vertices (in bytes): 11265
SimpleQuantizator(binary range partitioning packing) statistics:
- Bytes per triangle: 5.69
- Bytes per vertex: 11.29


In [8]:
encoder_radix_binary_tree_packing = SimpleQuantizator(pack_strip=Packing.RADIX_BINARY_TREE, verbose=True)
compressed_model = encoder_radix_binary_tree_packing.encode(bunny_model)

print_statistics(compressed_model, "SimpleQuantizator(radix binary tree packing)")

SimpleQuantizator verbose statistics:
- Header (in bytes): 32
- Triangles strip (in bytes): 16456
- Quantized vertices (in bytes): 11265
SimpleQuantizator(radix binary tree packing) statistics:
- Bytes per triangle: 5.59
- Bytes per vertex: 11.09
